In [ ]:
import pandas as pd
import numpy as np

pd.set_option('max_columns', 30)

In [ ]:
matching_features = pd.read_csv('2-matching_features_final.csv')
matched_data = pd.read_csv('3-matched_data_final.csv')


In [ ]:
matching_features.shape


In [ ]:
set(matched_data.columns).difference(set(matching_features.columns))


In [ ]:
matching_features


In [ ]:
matched_data


In [ ]:
matched_data.cbd_weight.value_counts()

In [ ]:
matched_data[matched_data['brand'] == 'plus'][['brand', 'strain_type']].drop_duplicates()

In [ ]:
matched_data.columns

In [ ]:
matched_data._id_product_match.value_counts()

In [ ]:
joined = matching_features.merge(matched_data, on = '_id')
joined = joined.drop([col for col in joined.columns if col.endswith('_y')], axis = 1)
joined


In [ ]:
joined.dtypes


In [ ]:
def develop_matched_data(matching_features_path: str) -> pd.DataFrame:
    """This function takes the path of the matching_features csv file as a string 
    and returns the matched_data as a Pandas dataframe."""
    
    matching_features = pd.read_csv(matching_features_path)
    
    matching_features['brand'] = matching_features['brand_name']
    matching_features['flavor'] = matching_features['flavor_ingestibles']
    matching_features['strain_type'] = matching_features['cannabis_strain_type']

    matching_features['product_name_line'] = np.nan
    
    matching_features['cbd_weight'] = matching_features['cbd_weight'] + ' cbd'
    matching_features['thc_weight'] = matching_features['thc_weight'] + ' thc'

    # get id_product_match from brand_name and flavor_ingestibles 
    matching_features['_id_product_match'] = (matching_features['brand_name'] + 
                                    '-' + 
                                    matching_features['flavor_ingestibles']
                                   ).str.replace(' ', '-').str.replace('\'', '')
    id_product_match_dict = {x: i for i, x in 
                             enumerate(set(matching_features['_id_product_match']))}
    matching_features = matching_features.replace({'_id_product_match': 
                                                   id_product_match_dict})

    matching_features = matching_features[['_id'
                                           , '_id_product_match'
                                           , 'brand'
                                           , 'flavor'
                                           , 'thc_weight'
                                           , 'cbd_weight'
                                           , 'strain_type'
                                           , 'day_report_date'
                                           , 'product_slug'
                                           , 'price'
                                           , 'data_source_name'
                                           , 'product_name_line']]
   
    return matching_features
    
    

In [ ]:
test = develop_matched_data('2-matching_features_final.csv')
test


In [ ]:
joined = test.merge(matching_features, on = '_id')
joined


In [ ]:
joined[joined['strain_type'] != joined['cannabis_strain_type']][
    ['_id', 'strain_type', 'cannabis_strain_type']]


In [ ]:
joined.dtypes

In [ ]:
def test_develop_matched_data():
    expected = pd.read_csv('3-matched_data_final.csv').drop(
        columns = ['_id_product_match', 'cbd_weight', 'strain_type']
    ).sort_values('_id').reset_index(drop = True)
    
    actual = develop_matched_data('2-matching_features_final.csv').drop(
        columns = ['_id_product_match', 'cbd_weight', 'strain_type']
    ).sort_values('_id').reset_index(drop = True)
    
    pd.testing.assert_frame_equal(expected, actual)
    

In [ ]:
test_develop_matched_data()